In [1]:
import operator
import os
import collections
import pickle
import autocomplete2
from collections import Counter
from bottle import route, run, debug
from autocomplete2 import predict
from autocomplete2 import models
from autocomplete2 import helpers
from autocomplete2 import split_predict
from autocomplete2 import predict_currword
from autocomplete2 import predict_second_word_given_first
import re 
import numpy as np 
import time
import csv 
import sys
from random import sample

In [2]:
# Python3 program to demonstrate auto-complete  
# feature using Trie data structure. 
class TrieNode(): 
    def __init__(self): 
          
        # Initialising one node for trie 
        self.children = {} 
        self.last = False
  
class Trie(): 
    def __init__(self): 
          
        # Initialising the trie structure. 
        self.root = TrieNode() 
        self.word_list = [] 
  
    def formTrie(self, keys): 
          
        # Forms a trie structure with the given set of strings 
        # if it does not exists already else it merges the key 
        # into it by extending the structure as required 
        for key in keys: 
            self.insert(key) # inserting one key to the trie. 
  
    def insert(self, key): 
          
        # Inserts a key into trie if it does not exist already. 
        # And if the key is a prefix of the trie node, just  
        # marks it as leaf node. 
        node = self.root 
        for a in list(key): 
        ### If the letter of the prefix is not found in the child nodes, add them to the trie ##
            if not node.children.get(a): 
                node.children[a] = TrieNode() 
            node = node.children[a] 
        node.last = True
  
    def search(self, key): 
          
        # Searches the given key in trie for a full match 
        # and returns True on success else returns False. 
        node = self.root 
        found = True
  
        for a in list(key): 
            if not node.children.get(a):  ### If the key is not found in the child nodes,  breaks the loop. 
                found = False
                break
  
            node = node.children[a]  ### Creates a new node with the letter not found in the child nodes.###
  
        return node and node.last and found 
  
    def suggestionsRec(self, node, word): 
          
        # Method to recursively traverse the trie 
        # and return a whole word.  
        if node.last: 
            self.word_list.append(word) 
        ## Add characters to the word in word list as found in the child nodes ###
        for a,n in node.children.items(): 
            self.suggestionsRec(n, word + a) ##### Adds the letters in the child nodes of the current node (node corresponding to the 
                                             #####last letter of the prefix) to the prefix and then appends it into the 
                                             ##### list of suggestions ####
    
    def printAutoSuggestions(self, key,WORDS,top_n): 
          
        # Returns all the words in the trie whose common 
        # prefix is the given key thus listing out all  
        # the suggestions for autocomplete. 
        self.word_list = []
        node = self.root 
        not_found = False
        temp_word = '' 
  
        for a in list(key): 
           ### If the key is not found in the children, break the loop ###
            if not node.children.get(a): 
                not_found = True
                break
           ### Add the keys to the temp_word string to form the prefix to be searched ###
            temp_word += a 
            ### Call the node corresponding to the current letter of the prefix if the character exists in the child nodes ###
            node = node.children[a] 
           #### Get the suggestions by calling the suggestionsRec function which gets the suggestions by looking at the child nodes of the node 
           #### corresponding to the last letter of the prefix ####
        self.suggestionsRec(node, temp_word)
        if not_found: 
            return 0
        elif node.last and not node.children: ### If the current node itself is the root node (no characters have been added to the trie 
            return -1                         ##structure)##
        #print(self.word_list)
        freq_dict  = Counter(WORDS)
        dict = {}
        for s in self.word_list:
            dict[s] = freq_dict[s]
        sorted_tup = sorted(dict.items(), key=operator.itemgetter(1),reverse = True)
        sorted_list = []
        for tup in sorted_tup[:top_n]:
            sorted_list.append(tup[0])
        return sorted_list

In [3]:
sys.getrecursionlimit()

3000

In [3]:
sys.setrecursionlimit(15000)

In [52]:
list = np.linspace(0,60,31)
print(list)

[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60.]


In [7]:
huge_file = "everything_combined.txt"

In [8]:
#### Creating a list of all words in the text file #####
huge_list = []
with open(huge_file, "r") as f:
    for line in f:
        huge_list.extend(line.split())

In [21]:
##### Creating a list of tuples of 2 consecutive words for autocompetion #####
keys = list(helpers.chunks(huge_list,2)) # keys to form the trie structure. 

In [20]:
keys[0]

['serosal', 'mass']

In [24]:
#### Combining the consecutive words into one string separated by an empty space and creating a new list ####
new_list = []
for i in range(0,len(keys)):
    curr_list = keys[i]
    new_list.append(curr_list[0]+ ' ' + curr_list[1])
print(new_list[3])

# creating trie object 
t = Trie() 

#### Creating the trie structure with the set of strings ####
start = time.time()
t.formTrie(new_list) 
end = time.time()
print("elapsed time=",end-start)

the descending
elapsed time= 20.345088958740234


In [27]:
#### For phrases of specific lengths ###
key = "th" # key for autocomplete suggestions.
status = ["Not found", "Found"] 
# autocompleting the given key using  
# our trie structure. 
start = time.time()
comp = t.printAutoSuggestions(key,new_list,top_n = 10) 
end  = time.time()
print(comp)
print("elapsed time  = ",end-start)
if comp == -1: 
    print("No other strings found with this prefix\n") 
elif comp == 0: 
    print("No string found with this prefix\n") 

['the patient', 'the left', 'the right', 'there is', 'the procedure', 'the ""patients""', 'the aortic', 'the setting', 'the chest', 'the mitral']
elapsed time  =  0.8671159744262695


In [28]:
#### Repeating for entire query lines #####
query_list = open("everything_combined.txt").readlines()
keys = query_list

# creating the trie structure with the  
# given set of strings.
t = Trie() 
start = time.time()
t.formTrie(query_list) 
end = time.time()
elapsed_time = end-start
print(elapsed_time)


34.659099817276


In [30]:
key = "lo" # key for autocomplete suggestions. 
# t.insert(key)
status = ["Not found", "Found"] 
start = time.time()
comp = t.printAutoSuggestions(key,keys,top_n = 10) 
print(comp)
end = time.time()
elapsed_time = end-start
print("Elapsed time=", elapsed_time)

['lower extremity bypass surgery discharge instructions\n', 'lower extremity\n', 'lower extremity edema .\n', 'lower leg\n', 'low back pain\n', 'low limb lead voltage .\n', 'lower limb\n', 'low lung volumes .\n', 'low lung volumes\n', 'locked-in state (G83.5)\n']
Elapsed time= 0.12334299087524414


In [ ]:
#### Attempting to store the formed trie data structure #####


In [31]:
##### Creating a list of elapsed times by creating the Trie data structure using distinct total number of queries ####
query_list = open("body_struct.txt").readlines()
#query_list = open("disorders.txt").readlines()
#query_list = open("procedures.txt").readlines()
#query_list = open("everything_combined.txt").readlines()
k = 0
elapsed_list_1 = []
while k<10:
    t = Trie() 
    start = time.time()
    t.formTrie(query_list) 
    end = time.time()
    elapsed_time = end-start
    elapsed_list_1.append(elapsed_time)
    k += 1
print(elapsed_list_1)


[16.681086778640747, 8.520795106887817, 10.436122179031372, 11.3989896774292, 9.251160144805908, 9.212028980255127, 11.252387046813965, 9.637441158294678, 9.280953168869019, 9.947311878204346]


In [44]:
#### Saving the list of elapsed times as a numpy array ###
insert_elapsed_list1 = elapsed_list_1
data = np.asarray(insert_elapsed_list1)
np.save('insert_elapsed_1.npy', data)

In [4]:
corpus = open("everything_combined.txt").readlines()

In [5]:
#### Randomly sampling queries and adding the mean elapsed time to the a list ####
mean_query_insert_list = []
for length in range(0,len(corpus),40000):
    query_list = sample(corpus,length)
    keys = query_list
    k = 0
    elapsed_list_1 = []
    while k<10:
        t = Trie() 
        start = time.time()
        t.formTrie(query_list) 
        end = time.time()
        elapsed_time = end-start
        elapsed_list_1.append(elapsed_time)
        k += 1
    mean_query_insert_list.append(np.mean(elapsed_list_1))
print(mean_query_insert_list)

[4.291534423828125e-07, 3.789134216308594, 7.469305205345154, 11.416516518592834, 15.322182130813598, 19.35610749721527, 24.497124791145325, 29.55403664112091, 34.563829112052915, 39.33153941631317, 44.23347852230072]


In [6]:
#### Saving the list of mean query insert times as a numpy array ###
data = np.asarray(mean_query_insert_list)
np.save('mean_query_insert_list_everything.npy', data)

In [8]:
#### Searching for autocomplete suggestions multiple times to create a list of elapsed times for different number of query lines #####
key = "abd" # key for autocomplete suggestions. 
status = ["Not found", "Found"] 
query_elapsed_list1 = []
k= 0 
while k<10:
    start = time.time()
    comp = t.printAutoSuggestions(key,keys,top_n = 10) 
    print(comp)
    end = time.time()
    elapsed_time = end-start
    query_elapsed_list1.append(elapsed_time)
    k += 1
print(query_elapsed_list1)
if comp == -1: 
    print("No other strings found with this prefix\n") 
elif comp == 0: 
    print("No string found with this prefix\n")

['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abdominopelvic', 'abdominoplasty', 'abdominoplastia', 'abdominla']
['abd', 'abdom', 'abdomen', 'abdomin', 'abdominal', 'abdominus', 'abd

In [17]:
#### Saving the elapsed time list for searching the trie data structure ####
data = np.asarray(query_elapsed_list1)
np.save('query_elapsed_list1.npy', data)

In [7]:
#huge_file = "body_struct.txt"
#huge_file = "combined_2.txt"
#huge_file = "combined_3.txt"
huge_file = "combined_processed.txt"
huge_list = []
with open(huge_file, "r") as f:
    for line in f:
        huge_list.extend(line.split())

# creating trie object
t = Trie()
# creating the trie structure with the  
# given set of strings.
start = time.time()
t.formTrie(huge_list) 
end = time.time()
elapsed_time = end-start
print(elapsed_time)

3.426222085952759


In [98]:
print(np.load('insert_single_time4.npy'))

[4.21788001 4.18371391 4.15810895 4.13986278 3.91943073 4.12861085
 4.24800205 3.98630404 4.02628303 4.47440886]


In [11]:
### Autocompletion for just one word ###
#huge_file = "body_struct.txt"
#huge_file = "combined_2.txt"
#huge_file = "combined_3.txt"
huge_file = "everything_combined.txt"
huge_list = []
with open(huge_file, "r") as f:
    for line in f:
        huge_list.extend(line.split())
# creating trie object 
# creating the trie structure with the  
# given set of strings.
insert_single_time1 = []
k = 0 
while k<1:
    t = Trie() 
    start = time.time()
    t.formTrie(huge_list) 
    end = time.time()
    elapsed_time = end-start
    insert_single_time1.append(elapsed_time)
    k +=1
print(insert_single_time1)

[5.228835105895996]


In [12]:
### Saving the list os elapsed times as a numpy array for different total number of words ####
data = np.asarray(insert_single_time1)
np.save('insert_single_time4.npy', data)

In [13]:
#### Measuring the query search time for single word ####
key = "A8" # key for autocomplete suggestions. 
keys = huge_list
status = ["Not found", "Found"]
query_single_list = []
k = 0 
while k<10:
    start = time.time()
    comp = t.printAutoSuggestions(key,keys,top_n = 10) 
    end = time.time() 
    elapsed_time = end-start
    query_single_list.append(elapsed_time)
    k += 1
print(comp)
#print(query_single_list)
if comp == -1: 
    print("No other strings found with this prefix\n") 
elif comp == 0: 
    print("No string found with this prefix\n") 

['A8', 'A80', 'A80-A89', 'A80-B09,', 'A80.0', 'A80.1', 'A80.2', 'A80.3', 'A80.30', 'A80.39']


In [100]:
#### Saving the list of elapsed times as a numpy array ###
data = np.asarray(query_single_list)
np.save('query_single_time4.npy', data)